<!-- Notebook Header Start -->

<h1 align="center">Running Kilosort & Analyzing Data</h1>

<p align="center">
  <strong>Author:</strong> Karl Bates<br>
  <strong>Date:</strong> 2024-11-18<br>
  <strong>Affiliation:</strong> Carnegie Mellon University, Cohen-Karni Lab  || Neuromechatronics Lab
</p>

---


## 📊 Notebook Outline

* **Importing binary files from SpikeInterface**
* **Running Kilosort spike sorting on the datasets of interest**
* **Importing the results of Kilosort back into Python for analysis / plotting**


## 📚 References & Additional Resources

- [Kilosort4 docs](https://github.com/MouseLand/Kilosort/tree/main)
- [SpikeInterface docs](https://github.com/SpikeInterface)

---

<!-- Notebook Header End -->


# 📈 Importing Libraries & Data
---

### import libraries

In [ ]:
# standard imports
from pathlib import Path
import os
import pandas as pd
import numpy as np
from kilosort import io
import matplotlib.pyplot as plt

# custom imports
from automations import RM1
from automations import SpikeInterface_wrapper
from automations import Kilosort_wrapper
from automations import plots

### filepath & probe definition

Using the spreadsheet, `Adapter_pinout.xlsx`, the contact ID's can be traced to the "device channel", and we can assign them on the probe. 

In this case, our channel indices correspond to the aux inputs to the intan headstage.

refer to the notebook, `RM1_pipeline.ipynb` within  the `dev_notebook` folder

In [2]:
# Path definitions
SAVE_DIRECTORY = Path(fr'D:\Data\CMU.80 Data\88 Analyzed Data\88.003 Initial Analysis, DW327')  # NOTE Specify the path where the data will be copied to, and where Kilosort4 results will be saved.
DATA_DIRECTORY = Path(fr'D:\Data\CMU.80 Data\82 External Data\82.002 Sample Rat Data from RM1 Project')  # NOTE Specify the path where the data will be copied to, and where Kilosort4 results will be saved.
# Create path if it doesn't exist
DATA_DIRECTORY.mkdir(parents=True, exist_ok=True)
SAVE_DIRECTORY.mkdir(parents=True, exist_ok=True)
PROBE_DIRECTORY = Path(r'D:\Data\CMU.80 Data\88 Analyzed Data\88.001 A1x32-Edge-5mm-20-177-A32\A1x32-Edge-5mm-20-177-A32.prb')

# 📈 Trial Summary
---

This is used to determine which trial is most likely to produce good units in a spike sorting analysis

### process data for import into Kilosort

I will use a combination of `SpikeInterface` & `Kilosort.io.extractors` to get the data I need

In [7]:
spikes = Kilosort_wrapper(SAVE_DIRECTORY,PROBE_DIRECTORY)

Preparing Kilosort wrapper...


In [ ]:
spikes.run_kilosort_trial_summary()

# ⏩ Import Kilosort Outputs (if they exist)

when kilosort is run, it creates a series of files saved next to the binary files.
use this method to import them. if there are no kilosort files, it will pass

In [ ]:
spikes.extract_kilosort_outputs()

array([18.716656, 26.29137 , 28.616701, ..., 18.148735, 20.001297,
       22.05708 ], dtype=float32)

In [17]:
spikes.kilosort_results["VF_8_241125_161626"]["channel_map.npy"]

array([14, 10,  5,  1, 11,  6,  4,  9, 12,  7,  3,  8, 13,  2, 15,  0, 16,
       31, 18, 29, 24, 25, 23, 17, 22, 30, 26, 27, 21, 28, 20, 19])

- **`results_dir`**: Constructs the path to where the Kilosort outputs are stored.
- **`ops`**: Loads the Kilosort options dictionary, which includes parameters and processing information.
- **`camps` & `contam_pct`**: Load cluster amplitudes and contamination percentages from TSV files.
- **`chan_map`**: Loads the mapping of electrode channels.
- **`templates`**: Loads the spike waveform templates.
- **`chan_best`**: Identifies the best (most representative) channel for each template by finding the channel with the maximum energy (sum of squared amplitudes).
- **`amplitudes`**: Loads the amplitudes of detected spikes.
- **`st`**: Loads spike times, typically in sample indices.
- **`clu`**: Loads cluster assignments for each spike.
- **`firing_rates`**: Calculates the firing rate for each unit (cluster) by counting the number of spikes and normalizing by the total recording time.
- **`dshift`**: Extracts the drift shift values from the options dictionary, which indicates the movement of the recording probe over time.

# 🐁 Analysis DRG Stimulation Trial on Rat DW323
---

### defining filepaths

In [ ]:
SAVE_DIRECTORY = Path(fr'D:\Data\CMU.80 Data\88 Analyzed Data\88.003 Initial Analysis, DW323')  # NOTE Specify the path where the data will be copied to, and where Kilosort4 results will be saved.
DATA_DIRECTORY = Path(fr'D:\Data\CMU.80 Data\82 External Data\82.002 Sample Rat Data from RM1 Project')  # NOTE Specify the path where the data will be copied to, and where Kilosort4 results will be saved.
# Create path if it doesn't exist
DATA_DIRECTORY.mkdir(parents=True, exist_ok=True)
SAVE_DIRECTORY.mkdir(parents=True, exist_ok=True)

In [ ]:
DW323 = RM1.Rat(DATA_DIRECTORY, PROBE_DIRECTORY, "DW323")
DW323.get_sc_data()

## 🔬 Exploratory Data Analysis (EDA)


see paper for more information. for this experiment, I elected to start my analysis with trial: 

**Rat: DW322 | Trial: DRGS_1_240918_125448**

In [ ]:
DW323.sc_data.keys()

In [ ]:
TRIAL_DRGS = "DRGS_11_240911_160638"
TRIAL_VF = ""

## 📑 metadata - DRGS

In [ ]:
DW323.drgs_experiment_notes

In [ ]:
DW323.drgs_trial_notes.loc[11].to_latex(r"D:\Data\CMU.80 Data\88 Analyzed Data\88.003 Initial Analysis, DW327\tables\drgs_trial_notes.tex",
                                        header=False,
                                        )
DW323.drgs_trial_notes.loc[11]

## 📑 metadata - DRGS